In [1]:
# process tart data for training

In [2]:
tart_path = '/home1/cxy/tart/ranker_train_full_contriever_minilm_denoised_w_instruction_unfollowing_10k_added_oqa_fixed/st_train_ranker_input.json'

In [3]:
import datasets
tart_data = datasets.load_dataset('json', data_files=tart_path, ignore_verifications=True)['train']

/home1/cxy/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset json (/home1/cxy/.cache/huggingface/datasets/json/default-ff83d81b6f54fc0a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:

# define tokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
import random
random.seed(42)
tokenizer = LlamaTokenizer.from_pretrained('/home1/cxy/alpaca-lora/llama-7b-hf')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [30]:
# count max document word length
from tqdm import tqdm
def count_max_doc_token_length(data):
    max_doc_token_length = 0
    for i in tqdm(range(len(data))):
        # doc_token_length = len(tokenizer(data[i]['document'], return_tensors='pt')['input_ids'][0])
        doc_token_length = len(data[i]['document'].split(' '))
        if doc_token_length > max_doc_token_length:
            max_doc_token_length = doc_token_length
    return max_doc_token_length

# max_doc_token_length = count_max_doc_token_length(tart_data)
# print(max_doc_token_length)

In [4]:
tart_data[0]

{'query': 'Find an evidence paragraph on the web with evidence and answer for this </s> what causes fistulas',
 'document': "web paragraph Dan Johnson. Echo Park is located in the heart of Dinosaur's canyon country. Here, the Yampa River flows into the Green River, which winds around the massive feature known as Steamboat Rock. The meeting of the two rivers along with nearby geologic faults created some of the monument's most dramatic scenery.",
 'label': 0}

In [32]:
# for each query, add an id, for each document, add another id
def add_ids(data, idx):
    data['id'] = idx
    return data
# tart_data = tart_data.map(add_ids, with_indices=True)
# tart_data[255]

In [33]:
# chunk the document into 256 word chunks, and add the question to each chunk, then evaluate the model on each chunk and take the chunk with the highest score as the representation of the document for the question

def chunk_examples(examples):
    chunks = []
    ids = []
    queries = []
    labels = []
    for sentence, label, query, tid in zip(examples["document"], examples["label"], examples["query"], examples["id"]):
        chunks += [' '.join(sentence.split(' ')[i:i + 256]) for i in range(0, len(sentence.split(' ')), 128)]
        for i in range(0, len(sentence.split(' ')), 128):
            ids.append(tid)
            queries.append(query)
            labels.append(label)
    return {"chunks": chunks, "ids":ids, "queries":queries, "labels":labels}
# chunked_dataset = tart_data.map(chunk_examples, batched=True, remove_columns=tart_data.column_names)

In [22]:
chunk_save_path ='/home1/cxy/alpaca-lora/process_data/chunked_tart_data'
chunked_dataset.save_to_disk(chunk_save_path)

Saving the dataset (0/10 shards):   0%|          | 0/4712048 [00:00<?, ? examples/s]

In [1]:
# evaluate the model on each chunk and take the chunk with the highest score as the representation of the document for the question
import sys
import os.path as osp

SRC_SUBDIR = '/home1/cxy/alpaca-lora/tart'
import os
print(os.getcwd())

if SRC_SUBDIR not in sys.path:
    print(f'Adding source directory to the sys.path: {SRC_SUBDIR!r}')
    sys.path.insert(1, SRC_SUBDIR)
from tart.modeling_enc_t5 import EncT5ForSequenceClassification
from tart.tokenization_enc_t5 import EncT5Tokenizer
import torch
import torch.nn.functional as F
from torch.nn import DataParallel
import numpy as np
# load TART full and tokenizer
model = EncT5ForSequenceClassification.from_pretrained("facebook/tart-full-flan-t5-xl")
tokenizer =  EncT5Tokenizer.from_pretrained("facebook/tart-full-flan-t5-xl")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = DataParallel(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

/home1/cxy/alpaca-lora/process_data
Adding source directory to the sys.path: '/home1/cxy/alpaca-lora/tart'
Using 8 GPUs


DataParallel(
  (module): EncT5ForSequenceClassification(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
               

In [2]:
import datasets
chunked_dataset = datasets.load_from_disk('/home1/cxy/alpaca-lora/process_data/chunked_tart_data')


In [14]:
chunked_dataset[2333]

{'chunks': "search snippets Spin City (TV Series 19962002) - IMDb  Comedy ... With Michael J. Fox, Charlie Sheen, Heather Locklear, Richard Kind. ... about the Winston administration's links to organized crime, Mike decides to take ..... In this sitcom, Deputy Mayor Mike Flaherty (Fox) and his City Hall staff must stop ... (Gaberman/Chaplin)the Mayor's Speech Writer, and Nikki Faber (Britton)...",
 'ids': 1509,
 'queries': 'Find the top 5 Web snippets that answer this </s> Mayor Randall Winston,Deputy Mayor Michael Flaherty,Nikki Faber',
 'labels': 1}

In [9]:
# replace </s> with <SEP> in the query
def replace_sep(examples):
    queries = []
    for query in examples["queries"]:
        queries.append(query.replace("</s>", "<SEP>"))
    return {"queries":queries, "chunks":examples["chunks"], "ids":examples["ids"], "labels":examples["labels"], "scores":examples["scores"]}
# chunked_dataset = chunked_dataset.map(replace_sep, batched=True, remove_columns=chunked_dataset.column_names)

In [16]:
chunked_dataset[2333]

{'chunks': "search snippets Spin City (TV Series 19962002) - IMDb  Comedy ... With Michael J. Fox, Charlie Sheen, Heather Locklear, Richard Kind. ... about the Winston administration's links to organized crime, Mike decides to take ..... In this sitcom, Deputy Mayor Mike Flaherty (Fox) and his City Hall staff must stop ... (Gaberman/Chaplin)the Mayor's Speech Writer, and Nikki Faber (Britton)...",
 'ids': 1509,
 'queries': 'Find the top 5 Web snippets that answer this <SEP> Mayor Randall Winston,Deputy Mayor Michael Flaherty,Nikki Faber',
 'labels': 1}

In [3]:
# evaluate the model on each chunk and take the chunk with the highest score as the representation of the document for the question
# model to device 5


def evaluate_model(examples):
    # encode
    features = tokenizer(examples["queries"], examples["chunks"],padding=True, truncation=True, return_tensors="pt")
    features = {k: v.to(device) for k, v in features.items()}
    with torch.no_grad():
        scores = model(**features).logits
        normalized_scores = [float(score[1]) for score in F.softmax(scores, dim=1)]
    #save the score of each chunk
    return {"scores":normalized_scores, "ids":examples["ids"], "labels":examples["labels"], "queries":examples["queries"], "chunks":examples["chunks"]}

# test_dataset = chunked_dataset.select(range(3600))
# test_dataset_scored = test_dataset.map(evaluate_model, batched=True, remove_columns=test_dataset.column_names, batch_size=1280)
chunked_dataset_scored = chunked_dataset.map(evaluate_model, batched=True, remove_columns=chunked_dataset.column_names, batch_size=1280)

Loading cached processed dataset at /home1/cxy/alpaca-lora/process_data/chunked_tart_data/cache-55caf8bb7d7372fa.arrow


In [4]:
# save chunked_dataset_scored
chunked_dataset_scored_save_path ='/home1/cxy/alpaca-lora/process_data/chunked_tart_data_scored'
chunked_dataset_scored.save_to_disk(chunked_dataset_scored_save_path)

In [5]:
chunked_dataset_scored = datasets.load_from_disk(chunked_dataset_scored_save_path)

In [10]:
chunked_dataset_scored = chunked_dataset_scored.map(replace_sep, batched=True, remove_columns=chunked_dataset_scored.column_names)

Map:   0%|          | 0/4712048 [00:00<?, ? examples/s]

In [24]:
chunked_dataset_scored[1]

{'chunks': 'To determine if the standard compliance check protocol is a valid measure of the experience of underage smokers when purchasing tobacco in unfamiliar communities. 160 tobacco outlets in eight Massachusetts communities where underage tobacco sales laws are vigorously enforced. Completed purchase rates were compared between underage smokers who behaved normally and inexperienced non-smoking youths who were not allowed to lie or present proof of age (ID). The "smoker protocol" increased the likelihood of a sale nearly sixfold over that for the non-smokers (odds ratio (OR) 5.7, 95% confidence interval (CI) 1.5 to 22). When the youths presented an ID with an underage birth date, the odds of a completed sale increased dramatically (OR 27, 95% CI 3.4 to 212). Clerks judged to be under 21 years of age were seven times more likely to make an illegal sale (OR 7.6, 95% CI 2.4 to 24.0). Commonly used compliance check protocols are too artificial to reflect accurately the experience of 

In [15]:
# for each ids get the chunk with the highest score
from collections import defaultdict
# for each id, the initial score is 0
id_score = defaultdict(float)
# save the idx of the chunk with the highest score
max_score_idx = defaultdict(int)
def get_max_score(example, idx):
    if example["scores"] > id_score[example["ids"]]:
        id_score[example["ids"]] = example["scores"]
        max_score_idx[example["ids"]] = idx
    return example
chunked_dataset_scored.map(get_max_score, with_indices=True)

Map:   0%|          | 0/4712048 [00:00<?, ? examples/s]

Dataset({
    features: ['chunks', 'ids', 'queries', 'labels', 'scores'],
    num_rows: 4712048
})

In [17]:
# get the chunk with the highest score
idx_list = []
for idx in max_score_idx.values():
    idx_list.append(idx)
print(len(idx_list))
chunked_dataset_scored_high = chunked_dataset_scored.select(idx_list)

2990000


In [54]:
chunked_dataset_scored_high[2897]

{'chunks': 'assistant to tell her what the problem was. She just didn\'t know how to fix it. She\'d been diagnosed with post-traumatic stress disorder, traced to a period of sexual abuse as a child and a life-threatening car accident. She\'d been hospitalized four times for the debilitating symptoms, and stress had caused bleeding ulcers bad enough to send her to the emergency room twice more. Along the way, she tried cognitive behavioral therapy, hypnosis and acupuncture. She tried an established therapy called eye movement desensitization and reprocessing, where a therapist used physical stimuli -- light tapping and guided eye movements -- to try to retrain her brain. It made her eyeballs feel like they would burst out of her head. She tried gestalt therapy, screaming out her rage. "Nothing worked," she says. "I got to the point where I just said, \'I\'m handicapped. I\'m just going to have to live my life like this.\' It was pretty horrible." A formal plan . More than 7 million Amer

In [26]:
# save chunked_dataset_scored_high
chunked_dataset_scored_high_save_path ='/home1/cxy/alpaca-lora/process_data/chunked_tart_data_scored_high'
chunked_dataset_scored_high.save_to_disk(chunked_dataset_scored_high_save_path)

Saving the dataset (0/6 shards):   0%|          | 0/2990000 [00:00<?, ? examples/s]

In [27]:
# process dev data
dev_path = '/home1/cxy/tart/ranker_train_full_contriever_minilm_denoised_w_instruction_unfollowing_10k_added_oqa_fixed/st_dev_ranker_input.json'

In [28]:
import datasets
dev_data = datasets.load_dataset('json', data_files=dev_path, ignore_verifications=True)['train']

/home1/cxy/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home1/cxy/.cache/huggingface/datasets/json/default-fa4895e1595f847b/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
dev_data[1]

{'query': 'You have to match this long sentence to a shorter compressed one </s> Andy Murray got back to winning ways by defeating Jesse Huta Galung of the Netherlands 6-4 6-4 in the first round of the ATP Marseille Open on Tuesday.',
 'document': 'short sentence Murray back to winning',
 'label': 1}

In [31]:
max_doc_len = count_max_doc_token_length(dev_data)
print(max_doc_len)

100%|██████████| 10000/10000 [00:00<00:00, 13127.19it/s]

1832


In [36]:
# add id
dev_data = dev_data.map(add_ids, with_indices=True)
dev_data[1]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'query': 'You have to match this long sentence to a shorter compressed one </s> Andy Murray got back to winning ways by defeating Jesse Huta Galung of the Netherlands 6-4 6-4 in the first round of the ATP Marseille Open on Tuesday.',
 'document': 'short sentence Murray back to winning',
 'label': 1,
 'id': 1}

In [43]:
# chunk the data
chunked_dev_data = dev_data.map(chunk_examples, batched=True, remove_columns=dev_data.column_names, batch_size=1280)
# score the chunks
chunked_dev_data_scored = chunked_dev_data.map(evaluate_model, batched=True, remove_columns=chunked_dev_data.column_names, batch_size=1280)
# replace </s> with <SEP> in the query
chunked_dev_data_scored = chunked_dev_data_scored.map(replace_sep, batched=True, remove_columns=chunked_dev_data_scored.column_names)
# get the chunk with the highest score
id_score = defaultdict(float)
max_score_idx = defaultdict(int)
chunked_dev_data_scored.map(get_max_score, with_indices=True)
idx_list = []
for idx in max_score_idx.values():
    idx_list.append(idx)
print(len(idx_list))
chunked_dev_data_scored_high = chunked_dev_data_scored.select(idx_list)
chunked_dev_data_scored_high[1]

Map:   0%|          | 0/15559 [00:00<?, ? examples/s]

10000


{'chunks': 'short sentence Murray back to winning',
 'ids': 1,
 'queries': 'You have to match this long sentence to a shorter compressed one <SEP> Andy Murray got back to winning ways by defeating Jesse Huta Galung of the Netherlands 6-4 6-4 in the first round of the ATP Marseille Open on Tuesday.',
 'labels': 1,
 'scores': 0.23419642448425293}

In [40]:
chunked_dev_data

Dataset({
    features: ['chunks', 'ids', 'queries', 'labels'],
    num_rows: 15559
})

In [41]:
dev_data

Dataset({
    features: ['query', 'document', 'label', 'id'],
    num_rows: 10000
})

In [42]:
chunked_dev_data_scored

Dataset({
    features: ['chunks', 'ids', 'queries', 'labels', 'scores'],
    num_rows: 15559
})

In [53]:
for i in range(100):
    print(chunked_dev_data_scored[i]["queries"])

Retrieve an paragraph-length answer to this question. <SEP> How accurate is a colorized photograph, and how does it work?
You have to match this long sentence to a shorter compressed one <SEP> Andy Murray got back to winning ways by defeating Jesse Huta Galung of the Netherlands 6-4 6-4 in the first round of the ATP Marseille Open on Tuesday.
I want to know the answer to the question. Can you find good evidence on the web? <SEP> is sundries group
Find the how-to article to achieve the following goal from WikihHow, a website collecting how-to articles. <SEP> Build Bleachers
Find a question that is duplicated with the following question asked in Quora, a community QA forum <SEP> What is the question that you've always wanted to ask the world?
Find an evidence from medical text book to answer this medical exam question <SEP> Ketone body formation takes place in:
WikiHow is an wiki-style community and database of how-to guides. Suggest the best article for the following <SEP> Prevent Renal

In [45]:
# save chunked_dev_data_scored_high
chunked_dev_data_scored_high_save_path ='/home1/cxy/alpaca-lora/process_data/chunked_dev_data_scored_high'
chunked_dev_data_scored_high.save_to_disk(chunked_dev_data_scored_high_save_path)

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]